In [2]:
import numpy as np

### Task 0: Markov Chain

In [11]:
"""Task 0: Markov Chain"""


def markov_chain(P, s, t=1):
    """Determines the probability of a markov chain being in a particular state
        after a specified number of steps
    Args:
        P: is a square 2D numpy.ndarray of shape (n, n) representing the
            transition matrix
            P[i, j] is the probability of transitioning from state i to state j
            n is the number of states in the markov chain
        s: is a numpy.ndarray of shape (1, n) representing the probability of
            starting in each state
        t: is the number of steps that the markov chain has been through
    Returns:
        a numpy.ndarray of shape (1, n) representing the probability of being
        in a specific state after t steps, or None on failure
    """
    for i in range(t):
        s = s @ P
    return s

In [12]:
P = np.array([[0.25, 0.2, 0.25, 0.3], [0.2, 0.3, 0.2, 0.3], [0.25, 0.25, 0.4, 0.1], [0.3, 0.3, 0.1, 0.3]])
s = np.array([[1, 0, 0, 0]])
print(markov_chain(P, s, 300))

[[0.2494929  0.26335362 0.23394185 0.25321163]]
